[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klavis-ai/klavis/blob/main/examples/openai/Use_Klavis_with_OpenAI.ipynb)


# OpenAI + Klavis AI Integration

This tutorial demonstrates how to build AI agents using OpenAI's function calling with Klavis MCP (Model Context Protocol) servers.


## Prerequisites

- **OpenAI API key** - Get at [openai.com](https://openai.com/)
- **Klavis API key** - Get at [klavis.ai](https://klavis.ai/)


In [1]:
# Install the required packages
%pip install -qU openai klavis requests



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
import json
from openai import OpenAI
from klavis import Klavis
from klavis.types import McpServerName, ConnectionType, ToolFormat

# Set environment variables
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"  # Replace with your actual OpenAI API key
os.environ["KLAVIS_API_KEY"] = "YOUR_KLAVIS_API_KEY"  # Replace with your actual Klavis API key

## Case Study : YouTube AI Agent

#### Step 1 - Create YouTube MCP Server using Klavis


In [7]:


klavis_client = Klavis(api_key=os.getenv("KLAVIS_API_KEY"))

youtube_mcp_instance = klavis_client.mcp_server.create_server_instance(
    server_name=McpServerName.YOUTUBE,
    user_id="1234",
    platform_name="Klavis",
    connection_type=ConnectionType.STREAMABLE_HTTP,
)

# print(f"🔗 YouTube MCP server created at: {youtube_mcp_instance.server_url}, and the instance id is {youtube_mcp_instance.instance_id}")


#### Step 2 - Create AI Agent to summarize YouTube video


In [18]:

youtube_tools = klavis_client.mcp_server.list_server_tools(
    server_url=youtube_mcp_instance.server_url,
    connection_type=ConnectionType.STREAMABLE_HTTP,
    format=ToolFormat.OPENAI,
)

openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def run_youtube_agent(video_url):
    """Run the YouTube AI agent to summarize a video."""
    messages = [
        {"role": "system", "content": "You are a comprehensive YouTube video analyzer. Use the available tools to analyze and summarize videos."},
        {"role": "user", "content": f"Provide a complete summary of this YouTube video with timestamp: {video_url}"}
    ]
    
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=youtube_tools.tools
    )
    
    print("response", response)
    
    message = response.choices[0].message
    
    if message.tool_calls:
        # Execute tool calls
        messages.append(message)
        
        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"🔧 Calling: {function_name}, with args: {function_args}")
            
            # Call the MCP tool
            result = klavis_client.mcp_server.call_server_tool(
                server_url=youtube_mcp_instance.server_url,
                tool_name=function_name,
                tool_args=function_args,
                connection_type=ConnectionType.STREAMABLE_HTTP
            )
            
            # Extract the actual content from the CallToolResult
            if result.success and result.result and result.result.content:
                content = result.result.content[0]['text']
            else:
                content = f"Error: {result.error}" if result.error else "No content returned"
            
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": content
            })
        
        # Get final response
        final_response = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        
        return final_response.choices[0].message.content
    
    return message.content

# Test with a YouTube video
YOUTUBE_VIDEO_URL = "https://www.youtube.com/watch?v=LCEmiRjPEtQ"  # pick a video you like!

summary = run_youtube_agent(YOUTUBE_VIDEO_URL)
print(f"\n📝 Video Summary:\n{summary}")


response ChatCompletion(id='chatcmpl-BkaclCk6j6lGKZSdeuqbc10NBH3Mn', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_A4Kllzb9i2nL5zml62ZRhmkT', function=Function(arguments='{"url":"https://www.youtube.com/watch?v=LCEmiRjPEtQ"}', name='get_youtube_video_transcript'), type='function')]))], created=1750443571, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=CompletionUsage(completion_tokens=33, prompt_tokens=161, total_tokens=194, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
🔧 Calling: get_youtube_video_transcript, with args: {'url': 'https://www.

✅ Great! You've successfully created an AI agent that uses OpenAI function calling with Klavis MCP servers to summarize YouTube videos!

## Summary

This tutorial showed how to integrate OpenAI with Klavis AI:

1. **Created YouTube MCP Server** using Klavis
2. **Set up OpenAI Function Calling** with MCP Server
3. **Built AI Agent** that summarizes YouTube videos

The combination of OpenAI's powerful language models with Klavis MCP servers provides seamless access to external tools and services. Happy building! 🚀
